# Computing hyperbolic magnitudes

[Implementation](https://github.com/jlvdb/hyperbolic) of Luption et al. (1999) by Jan Luca van den Busch.

Hyperbolic magnitudes aim to overcome limitations of classical magnitudes, which are logarithmic in flux. Hyperbolic magnitudues are implemented using the inverse hyperbolic sine and therefore have a linear behaviour in flux at low signal to noise, which gradually transitions to the classical logarithmic scaling at high signal to noise (i.e. equivalent to classical magnitudes in this limit).

This notebooks provides an example of how to convert classical to hyperbolical magnitudes using the pipeline stages `HyperbolicSmoothing` and `HyperbolicMagnitudes`in the `rail.core` module.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import hyperbolic

import rail
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilPhotometry import HyperbolicSmoothing, HyperbolicMagnitudes

We first set up a data store for interactive usage of RAIL (see the `rail/examples/goldenspike/goldenspike.ipynb` for further examples).

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

Next we load some DC2 sample data that provides LSST ugrizy magnitudes and magnitude errors, which we want to convert to hyperbolic magnitudes.

In [ ]:
RAIL_DIR = os.path.dirname(rail.__file__)
testFile = os.path.join(RAIL_DIR, '../tests/data/test_dc2_training_9816.pq')
test_mags = DS.read_file("test_data", TableHandle, testFile)

## Determining the smoothing parameters

First we run the `rail.core.HyperbolicSmoothing` stage. This stage computes the smoothing parameter (called $b$ in Lupton et al. 1999), which determines the transition between the linear and logarithmic behaviour of the hyperbolic magnitudes.

The **input** for this stage is a table containing magnitudes and magnitude errors per object. In this example, we assume that the magnitude zeropoint is 0.0 and that we want to convert all 6 LSST bands. This can be specified with the `magnitude_columns` and `magerror_columns` parameters, which list the names of the magnitude columns and their corresponding magnitude errors.

In [ ]:
lsst_bands = 'ugrizy'
coldefs = dict(
    magnitude_columns=[f"mag_{band}_lsst" for band in lsst_bands],
    magerror_columns=[f"mag_err_{band}_lsst" for band in lsst_bands])

smooth = HyperbolicSmoothing.make_stage(
    name='hyperbolic_smoothing',
    zeropoint=0.0, **coldefs)
smooth.compute(test_mags)

The **output** of this stage is a table of relevant statistics required to compute the hyperbolic magnitudes per filter:
- the median flux error
- the zeropoint (which can be computed by comparing fluxes and magnitudes in the original `hyperbolic` code)
- the reference flux $f_{\rm ref}$ that corresponds to the given zeropoint
- the smoothing parameter $b$ (in terms of the absolute and the relative flux $x = f / f_{\rm ref}$

The `field ID` column is currently not used by the RAIL module and can be ignored.

In [ ]:
smooth_params = smooth.get_handle("parameters").data
smooth_params

## Computing the magnitudes

Based on the smoothing parameters, the hyperbolic magnitudes are computed with be computed by `rail.core.HyperbolicMagnitudes`.

The **input** for this module is, again, the table with magnitudes and magnitude errors and the output table of `rail.core.HyperbolicSmoothing`, as well as the column names as defined above.

In [ ]:
hypmag = HyperbolicMagnitudes.make_stage(
    name='hyperbolic_magnitudes',
    **coldefs)
hypmag.compute(test_mags, smooth_params)

The **output** of this module is a table with hyperbolic magnitudes and their corresponding error.

**Note:** The current default is to relabel the columns names by substituting `mag_` by `mag_hyp_`. If this substitution is not possible, the column names are identical to the input table with classical magnitudes.

In [ ]:
test_hypmags = hypmag.get_handle("output").data
test_hypmags

## Some check plots

In [ ]:
filt = "r"

mag_class = test_mags.data[f"mag_{filt}_lsst"]
magerr_class = test_mags.data[f"mag_err_{filt}_lsst"]
mag_hyp = test_hypmags[f"mag_hyp_{filt}_lsst"]
magerr_hyp = test_hypmags[f"mag_hyp_err_{filt}_lsst"]
# flux_class = np.exp((0.0 - mag_class) / hyperbolic.pogson)
# fluxerr_class = magerr_class / hyperbolic.pogson * flux_class
# SN = flux_class / fluxerr_class

fig = plt.figure(dpi=100)
plt.axhline(y=0.0, color="k", lw=0.55)
plt.scatter(mag_class, mag_class - mag_hyp, s=1)
plt.xlabel("Classical magnitudue")
plt.ylabel("Classical $-$ hyperbolic magnitude")